<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Create_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##DataSet

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists

#Preprocess

In [10]:
df_text=pd.read_csv('/content/drive/MyDrive/PaperA3/TFIDF/L2_G08')

In [11]:
path=[["MultipartiteRank","/content/drive/MyDrive/PaperA3/MultipartiteRank/MultipartiteRank_20_"],
      ["YAKE","/content/drive/MyDrive/PaperA3/YAKE/YAKE_20_"],
      ["KeyBERT","/content/drive/MyDrive/PaperA3/KeyBERT/KeyBERT_20_"],
      ["Key2Vec","/content/drive/MyDrive/PaperA3/Key2Vec/Key2Vec_20_"],
      ["PositionRank","/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_"]]

In [21]:
for j in range(len(path)) :
    df_remain=pd.DataFrame(columns = ['label','Text'])
    for i in range(1,80):
      filename=path[j][1]+str(i)+".csv"
      
      if (exists(filename)==True):
              
              df2=pd.read_csv(filename)
              df2=df2[['label','Text']]
              
              df_remain=pd.concat([df_remain, df2])
      df_remain.label=df_remain.label.astype(float).astype(int) 
      df_remain.drop_duplicates(subset=['label','Text'], keep='last')       
    print(path[j][0],len(df_remain),len(df_remain.label.unique().tolist()))
    df_remain.to_csv(path[j][1]+'Total')

MultipartiteRank 26029 25926
YAKE 26927 25926
KeyBERT 25923 25921
Key2Vec 27111 25924
PositionRank 26123 25924


#Diagnoses_labels

In [22]:

df_diagnosis=pd.read_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Final_L1.csv')

In [23]:
filename=path[0][1]+'Total'
df_text=pd.read_csv(filename)
print(path[0][0])

MultipartiteRank


In [24]:
print(df_text.shape)
print(df_diagnosis.shape)

(26029, 3)
(52307, 21)


In [25]:
df_text.head()

,Unnamed: 0,label,Text
0,0,157267,"ulcer, ankle edema, pain, finding, hodgkin dis..."
1,0,123103,"heart valve disease, cholesterol measurement t..."
2,1,139931,"mitral valve insufficiency, hypertensive disea..."
3,2,198295,"hypertensive disease, edema, cholecystitis, ox..."
4,3,177962,"vessel coronary disease, left ventricular func..."


#تولید دیکشنری از مفاهیم

In [26]:
dd=pd.read_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Dictionary.csv')
dictionary = dict(zip(dd.first_cuid, dd.canonical_name))

# تولید دیکشنری

In [ ]:
filename="/content/drive/MyDrive/PaperA3/CUI/CUI_Total.csv"
df=pd.read_csv(filename)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
dd=df[['first_cuid', 'canonical_name']]
print(len(dd))
dd.drop_duplicates(keep="last",inplace=True)
print(len(dd))

18075318
25312


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:

dd.head()

,first_cuid,canonical_name
355,C1278302,Hepatitis B screening test
400,C0004285,Audiology
833,C0001916,Albinism
1130,C0013069,Double Outlet Right Ventricle
1691,C1394030,Coronal hypospadias


برای تولید دیکشنری به این فرم حتما باید زیپ کرد

In [ ]:

dictionary = dict(zip(dd.first_cuid, dd.canonical_name))


In [ ]:
dictionary

In [ ]:
dd.to_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Dictionary.csv')


#Integration

#تولید دیتاست

In [27]:
import ast
def complex_function( ini_list):

    res = ast.literal_eval(ini_list)
    tms_text=''
    try:
      for i in res:
        
        if(i.upper().startswith("NOT")):
          tms_text=tms_text +'NOT '+dictionary[i[4:].upper()] +' '
        else:
          tms_text=tms_text +dictionary[i.upper()] +' '  
    except :
          print(i)    
    return tms_text 

In [28]:
dictionary['C0026727']

'Mucous body substance'

In [ ]:
#only for TFIDF
df_text['Text'] = df_text.apply(lambda x: complex_function(x['list_CUIs']), axis=1)
df_text.head()


In [31]:
df_text = df_text.rename(columns={'label': 'HADM_ID'})

In [32]:
df_total = pd.merge(df_text, df_diagnosis, on=["HADM_ID"],how="left")
print(df_total.shape)
df_total.head()

(26029, 23)


,Unnamed: 0_x,HADM_ID,Text,Unnamed: 0_y,L1,1,2,3,4,5,...,9,10,11,12,13,14,15,16,17,18
0,0,157267,"ulcer, ankle edema, pain, finding, hodgkin dis...",29918.0,13.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,123103,"heart valve disease, cholesterol measurement t...",12144.0,18.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,139931,"mitral valve insufficiency, hypertensive disea...",20902.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,198295,"hypertensive disease, edema, cholecystitis, ox...",51440.0,25.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3,177962,"vessel coronary disease, left ventricular func...",40804.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Only for TFIDF
df_total.drop(['L1','Unnamed: 0_x',  'TFIDF', 'list_CUIs', 'number_CUIs','Unnamed: 0_y'], axis=1, inplace=True)
df_total.columns



In [34]:
df_total.drop(['L1','Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=True)
df_total.columns

Index(['HADM_ID', 'Text', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18'],
      dtype='object')

In [35]:

df_raw=df_total
df_raw = df_raw.rename(columns={'Text': 'comment_text'})

In [36]:
df_raw.shape

(26029, 20)

In [37]:
def bool(value):    
  if(value==1):
          tms_text=True
  else:
          tms_text=False    
  return tms_text 




In [38]:
df_raw.rename(columns={'1' : 'L_1.0', '2':'L_2.0', '3':'L_3.0', '4':'L_4.0', '5':'L_5.0', '6':'L_6.0', '7':'L_7.0', '8':'L_8.0', '9':'L_9.0', '10':'L_10.0',
       '11':'L_11.0', '12':'L_12.0', '13':'L_13.0', '14':'L_14.0', '15':'L_15.0', '16':'L_16.0', '17':'L_17.0', '18':'L_18.0'}, inplace=True)
df_raw.columns

Index(['HADM_ID', 'comment_text', 'L_1.0', 'L_2.0', 'L_3.0', 'L_4.0', 'L_5.0',
       'L_6.0', 'L_7.0', 'L_8.0', 'L_9.0', 'L_10.0', 'L_11.0', 'L_12.0',
       'L_13.0', 'L_14.0', 'L_15.0', 'L_16.0', 'L_17.0', 'L_18.0'],
      dtype='object')

In [39]:
df_raw['L_1.0'] = df_raw.apply(lambda x: bool(x['L_1.0']), axis=1)  
df_raw['L_2.0'] = df_raw.apply(lambda x: bool(x['L_2.0']), axis=1)  
df_raw['L_3.0'] = df_raw.apply(lambda x: bool(x['L_3.0']), axis=1)  
df_raw['L_4.0'] = df_raw.apply(lambda x: bool(x['L_4.0']), axis=1) 
df_raw['L_5.0'] = df_raw.apply(lambda x: bool(x['L_5.0']), axis=1)  
df_raw['L_6.0'] = df_raw.apply(lambda x: bool(x['L_6.0']), axis=1)  
df_raw['L_7.0'] = df_raw.apply(lambda x: bool(x['L_7.0']), axis=1)  
df_raw['L_8.0'] = df_raw.apply(lambda x: bool(x['L_8.0']), axis=1)  
df_raw['L_9.0'] = df_raw.apply(lambda x: bool(x['L_9.0']), axis=1) 
df_raw['L_10.0'] = df_raw.apply(lambda x: bool(x['L_10.0']), axis=1)  
df_raw['L_11.0'] = df_raw.apply(lambda x: bool(x['L_11.0']), axis=1)  
df_raw['L_12.0'] = df_raw.apply(lambda x: bool(x['L_12.0']), axis=1)  
df_raw['L_13.0'] = df_raw.apply(lambda x: bool(x['L_13.0']), axis=1)  
df_raw['L_14.0'] = df_raw.apply(lambda x: bool(x['L_14.0']), axis=1) 
df_raw['L_15.0'] = df_raw.apply(lambda x: bool(x['L_15.0']), axis=1)  
df_raw['L_16.0'] = df_raw.apply(lambda x: bool(x['L_16.0']), axis=1)  
df_raw['L_17.0'] = df_raw.apply(lambda x: bool(x['L_17.0']), axis=1)  
df_raw['L_18.0'] = df_raw.apply(lambda x: bool(x['L_18.0']), axis=1)  


In [40]:
df_raw.to_csv('/content/drive/MyDrive/PaperA3/Final_Dataset/df_raw_'+ path[0][0]+".csv")